# 🎙️ Qwen3-TTS API Server v4

Voice Cloning TTS on Colab GPU.

In [ ]:
!pip install -q qwen-tts flask flask-cors pyngrok soundfile numpy

In [ ]:
import torch
from qwen_tts import Qwen3TTSModel

print("🔄 Loading Qwen3-TTS model...")
print(f"   GPU: {torch.cuda.get_device_name(0)}")

model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
)

print("✅ Model loaded!")

In [ ]:
# Enter ngrok token: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_TOKEN = ""  # @param {type:"string"}

from pyngrok import ngrok
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ ngrok token set!")

In [ ]:
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
import soundfile as sf
import io, base64, tempfile, os

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "model": "Qwen3-TTS-1.7B", "gpu": torch.cuda.get_device_name(0)})

@app.route('/api/tts', methods=['POST'])
def generate_tts():
    try:
        data = request.json
        text = data.get('text', 'Hello')
        language = data.get('language', 'English')
        ref_audio_b64 = data.get('ref_audio')
        ref_text = data.get('ref_text', '')

        print(f"🎙️ Gen: lang={language}, chars={len(text)}, clone={bool(ref_audio_b64)}, ref_text={bool(ref_text)}")

        if ref_audio_b64:
            audio_bytes = base64.b64decode(ref_audio_b64)
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as f:
                f.write(audio_bytes)
                temp_path = f.name
            try:
                # If ref_text is provided, use full ICL mode, otherwise use x_vector_only
                if ref_text and ref_text.strip():
                    wavs, sr = model.generate_voice_clone(
                        text=text,
                        language=language,
                        ref_audio=temp_path,
                        ref_text=ref_text,
                    )
                else:
                    # Use x_vector only mode (no ref_text needed)
                    wavs, sr = model.generate_voice_clone(
                        text=text,
                        language=language,
                        ref_audio=temp_path,
                        x_vector_only_mode=True,
                    )
            finally:
                os.unlink(temp_path)
        else:
            # No reference audio - use default voice
            wavs, sr = model.generate_voice_clone(
                text=text,
                language=language,
                ref_audio="https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-TTS-Repo/clone.wav",
                x_vector_only_mode=True,
            )

        buffer = io.BytesIO()
        sf.write(buffer, wavs[0], sr, format='WAV')
        buffer.seek(0)
        print(f"✅ Generated {len(wavs[0])/sr:.2f}s")
        return send_file(buffer, mimetype='audio/wav', as_attachment=True, download_name='output.wav')
    except Exception as e:
        import traceback; traceback.print_exc()
        return jsonify({"error": str(e)}), 500

@app.route('/api/clone', methods=['POST'])
def voice_clone():
    return generate_tts()

public_url = ngrok.connect(5000)
print("\n" + "="*50)
print(f"🚀 URL: {public_url}")
print("="*50 + "\n")

app.run(port=5000)